In [1]:
import json
import pandas as pd

In [29]:
f = open('/home/dy/filerepo/Research/multiagent-human-experiment/mturk/experiment_data/mturk-experiment-5-20210827-bonus-flipped.json')
data = json.load(f)
f.close()

In [30]:
feedback_subtasks = [5, 7, 9, 11]
game_subtasks = [4, 6, 8, 10]
workers = range(len(data))
agents = [game[1] for game in data[0]['results']['data'][-1]['games']]
print(agents)

['sac_self_play_coordination_ring_0', 'nct_coordination_ring', 'action_cml_v_sac_none_no_ib', 'sac_v_bc_states_coordination_ring_0']


In [31]:
def get_subtask(agent, worker, game_subtasks, data):

    for game_subtask in game_subtasks: #find the right subtask for the agent order
        # if (data[worker]['results']['data'][game_subtask]['data']['trajectory'][0]['player_1_id'] == agent):
        if (data[worker]['results']['data'][game_subtask]['params']['playerOne'] == agent):
            return game_subtask
    return -1

def get_reward(worker, game_subtask, data):
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    total_reward = 0

    # for timestep in data[worker]['results']['data'][game_subtask]['data']['trajectory']:
    for timestep in game_data['trajectory']:
        total_reward += timestep['reward']

    return total_reward

def get_actions(worker, game_subtask, data): #return actions per second
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    count = 0
    STAY_ACTION = [0, 0]
    for i in range(len(game_data['trajectory'])):
        if(game_data['trajectory'][i]['joint_action'][0] != STAY_ACTION):
            count += 1

    return count / 60

def get_actions_no_repeats(worker, game_subtask, data): #return actions per second
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    count = 0
    STAY_ACTION = [0, 0]
    prev_action = None
    for i in range(len(game_data['trajectory'])):
        curr_action = game_data['trajectory'][i]['joint_action'][0]
        if(curr_action != STAY_ACTION and curr_action != prev_action):
            count += 1
        prev_action = curr_action

    return count / 60


In [32]:
score_data = pd.DataFrame()

all_scores = []
for worker in workers:
    scores_dict = {}
    for agent in agents:
        game_subtask = get_subtask(agent, worker, game_subtasks, data)
        reward = get_actions_no_repeats(worker, game_subtask, data)
        scores_dict[agent] = reward
        print(worker, agent, reward)
    all_scores.append(scores_dict)

0 sac_self_play_coordination_ring_0 1.6
0 nct_coordination_ring 1.5
0 action_cml_v_sac_none_no_ib 0.5
0 sac_v_bc_states_coordination_ring_0 1.4
1 sac_self_play_coordination_ring_0 1.2
1 nct_coordination_ring 1.2
1 action_cml_v_sac_none_no_ib 1.0
1 sac_v_bc_states_coordination_ring_0 1.1666666666666667
2 sac_self_play_coordination_ring_0 2.316666666666667
2 nct_coordination_ring 2.2
2 action_cml_v_sac_none_no_ib 2.1333333333333333
2 sac_v_bc_states_coordination_ring_0 2.066666666666667
3 sac_self_play_coordination_ring_0 2.05
3 nct_coordination_ring 1.85
3 action_cml_v_sac_none_no_ib 1.7333333333333334
3 sac_v_bc_states_coordination_ring_0 2.6166666666666667
4 sac_self_play_coordination_ring_0 3.75
4 nct_coordination_ring 3.3333333333333335
4 action_cml_v_sac_none_no_ib 1.7666666666666666
4 sac_v_bc_states_coordination_ring_0 3.65
5 sac_self_play_coordination_ring_0 1.7166666666666666
5 nct_coordination_ring 3.3666666666666667
5 action_cml_v_sac_none_no_ib 3.1666666666666665
5 sac_v_bc_

In [34]:
actions_df = pd.DataFrame(all_scores)
actions_df['Total Average'] = actions_df.mean(axis=1)
actions_df

,sac_self_play_coordination_ring_0,nct_coordination_ring,action_cml_v_sac_none_no_ib,sac_v_bc_states_coordination_ring_0,Total Average
0,1.600000,1.500000,0.500000,1.400000,1.250000
1,1.200000,1.200000,1.000000,1.166667,1.141667
2,2.316667,2.200000,2.133333,2.066667,2.179167
3,2.050000,1.850000,1.733333,2.616667,2.062500
4,3.750000,3.333333,1.766667,3.650000,3.125000
5,1.716667,3.366667,3.166667,2.300000,2.637500
6,2.166667,1.833333,0.950000,1.466667,1.604167
7,3.916667,3.550000,3.333333,4.400000,3.800000
8,0.650000,0.750000,0.966667,0.716667,0.770833
9,2.533333,1.700000,0.516667,2.366667,1.779167


In [35]:
actions_df['Total Average'].mean()

2.2683333333333335

In [16]:
worker = 1
agent = 'action_cml_v_sac_none_no_ib'
game_subtask = get_subtask(agent, worker, game_subtasks, data)

game_id = data[worker]['results']['data'][game_subtask]['trial_id']

f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
game_data = json.load(f)    
f.close()

count = 0
STAY_ACTION = [0, 0]
for i in range(len(game_data['trajectory'])):
    if(game_data['trajectory'][i]['joint_action'][0] != STAY_ACTION):
        count += 1
        print(game_data['trajectory'][i]['joint_action'][0])

[-1, 0]
[0, 1]
[-1, 0]
interact
[0, -1]
[0, -1]
[1, 0]
[0, -1]
interact
[-1, 0]
[0, 1]
[0, 1]
[0, 1]
interact
[0, -1]
[0, -1]
[1, 0]
[-1, 0]
[1, 0]
[1, 0]
[-1, 0]
[1, 0]
[1, 0]
interact
[-1, 0]
[-1, 0]
[-1, 0]
[0, 1]
[0, 1]
[0, 1]
interact
[0, -1]
[0, -1]
[0, -1]
[1, 0]
[1, 0]
[-1, 0]
[1, 0]
[1, 0]
interact
[-1, 0]
[-1, 0]
[-1, 0]
[0, 1]
[0, 1]
[0, 1]
interact
[0, -1]
[0, -1]
[0, -1]
[1, 0]
[1, 0]
interact
[-1, 0]
[0, -1]
interact
[1, 0]
interact
[0, 1]
[0, 1]
[0, 1]
[-1, 0]
[0, 1]
interact
[-1, 0]
[0, 1]
interact
[0, -1]
[0, -1]
[0, -1]
[1, 0]
[1, 0]
interact
[-1, 0]
[-1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[-1, 0]
[0, -1]
[-1, 0]
[-1, 0]
[-1, 0]
[0, 1]
[0, 1]
[0, 1]
interact
[0, -1]
[1, 0]
[0, 1]
[1, 0]
[1, 0]
[1, 0]
[0, -1]
[0, -1]
[0, -1]
[1, 0]
interact
[-1, 0]
[-1, 0]
[-1, 0]
[0, 1]
[-1, 0]
interact
interact
[0, -1]
[1, 0]
[-1, 0]
[0, -1]
[1, 0]
[0, -1]
interact
[-1, 0]
[0, 1]
[0, -1]
[0, 1]
[0, 1]
[0, 1]
interact
[0, -1]
[0, -1]
[0, -1]
[1, 0]
[1, 0]
interact
[-1, 0]
[-1, 0]
[1, 0]
[

In [26]:
def get_actions_no_repeats(worker, game_subtask, data): #return actions per second
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    count = 0
    STAY_ACTION = [0, 0]
    prev_action = None

    for i in range(len(game_data['trajectory'])):
        curr_action = game_data['trajectory'][i]['joint_action'][0]
        if(curr_action != STAY_ACTION and curr_action != prev_action):
            count += 1
        print(game_data['trajectory'][i]['joint_action'][0])
        prev_action = curr_action

    return count / 60


worker = 1
agent = 'action_cml_v_sac_none_no_ib'
game_subtask = get_subtask(agent, worker, game_subtasks, data) #get subtask number from agent name
print(game_subtask)
apm = get_actions_no_repeats(worker, game_subtask, data)

6
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
interact
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
interact
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
interact
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 

In [28]:
worker = 1
data[worker]['results']['data'][4]['params']['playerOne']

'sac_self_play_coordination_ring_0'